In [1]:
import sys
sys.path.append("../src/")

In [2]:
from plugin_write_and_run import *

In [3]:
%%write_and_run ../src/main.py
import numpy as np
import tensorflow as tf
from tensorflow.keras import optimizers as opt
import random
import time
import json
import os
import sys
sys.path.append("../src")
from config import *
from make_env import *
from replay_buffer import *
from networks import *
from agent import *
from super_agent import *

In [4]:
env = make_env("simple_adversary")

In [5]:
super_agent = SuperAgent(env)

In [7]:
if PATH_LOAD_FOLDER is not None:
    print("loading weights")
    actors_state = env.reset()
    actors_action = super_agent.get_actions([actors_state[index][None, :] for index in range(super_agent.n_agents)])
    [super_agent.agents[index].target_actor(actors_state[index][None, :]) for index in range(super_agent.n_agents)]
    state = np.concatenate(actors_state)
    actors_action = np.concatenate(actors_action)
    [super_agent.agents[index].critic(state[None, :], actors_action[None, :]) for index in range(super_agent.n_agents)]
    [super_agent.agents[index].target_critic(state[None, :], actors_action[None, :]) for index in range(super_agent.n_agents)]
    super_agent.load()

    print(super_agent.replay_buffer.buffer_counter)
    print(super_agent.replay_buffer.n_games)

loading weights
125025
0


In [8]:
scores = []

for i in range(MAX_GAMES):
    actors_state = env.reset()
    done = [False for i in range(super_agent.n_agents)]
    score = 0
    step = 0
    while not any(done):
        actors_action = super_agent.get_actions([actors_state[index][None, :] for index in range(super_agent.n_agents)])
        actors_next_state, reward, done, info = env.step(actors_action)
        
        state = np.concatenate(actors_state)
        next_state = np.concatenate(actors_next_state)
        
        super_agent.replay_buffer.add_record(actors_state, actors_next_state, actors_action, state, next_state, reward, done)
        
        actors_state = actors_next_state
        
        score += sum(reward)
        step += 1
        if step >= 25:
            break
    
    if super_agent.replay_buffer.check_buffer_size():
        super_agent.train()
    
    scores.append(score)
    avg_score = np.mean(scores[-100:])
    if i % EVALUATION_FREQUENCY == 0 and super_agent.replay_buffer.check_buffer_size():
        print(avg_score)
    if i % SAVE_FREQUENCY == 0 and super_agent.replay_buffer.check_buffer_size():
        super_agent.save()

-32.535965435220724
-7.897232697842387
-4.614325009343534


KeyboardInterrupt: 